In [ ]:
# Fix FutureWarning Messages in scikit-learn
# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime, timedelta
from dateutil.parser import parse as dt_parse
from collections import Counter

from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error

import xgboost as xgb

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Step 0) Check training file.

In [2]:
!head /kaggle/input/chh-ola/train.csv

ID,vendor+AF8-id,pickup+AF8-loc,drop+AF8-loc,driver+AF8-tip,mta+AF8-tax,distance,pickup+AF8-time,drop+AF8-time,num+AF8-passengers,toll+AF8-amount,payment+AF8-method,rate+AF8-code,stored+AF8-flag,extra+AF8-charges,improvement+AF8-charge,total+AF8-amount
0,1,170,233,1.83,0.5,0.7,04/04/2017 05:59:43 PM,04/04/2017 06:05:04 PM,1,0,1,1,N,1,0.3,9.13
1,2,151,243,3.56,0.5,4.64,04/03/2017 07:03:34 PM,04/03/2017 07:20:04 PM,1,0,1,1,N,1,0.3,21.36
2,2,68,90,1.5,0.5,1.29,04/03/2017 03:06:13 PM,04/03/2017 03:12:30 PM,2,0,1,1,N,0,0.3,8.8
3,2,142,234,1.5,0.5,2.74,04/04/2017 08:10:52 AM,04/04/2017 08:27:00 AM,1,0,1,1,N,0,0.3,14.8
4,2,238,238,0,0.5,0.45,04/05/2017 02:02:59 PM,04/05/2017 02:05:41 PM,6,0,2,1,N,0,0.3,4.8
5,1,230,48,1.05,0.5,0.4,04/03/2017 09:10:13 AM,04/03/2017 09:12:09 AM,1,0,1,1,N,0,0.3,5.35
6,2,236,140,2.46,0.5,1.72,04/03/2017 12:04:52 PM,04/03/2017 12:20:54 PM,3,0,1,1,N,0,0.3,14.76
7,1,236,13,5.95,0.5,8.8,04/04/2017 03:19:38 PM,04/04/2017 03:48:18 PM,1,0,1,1,N,0,0.3,35.75
8,1,229,141,0,

In [4]:
df = pd.read_csv('/kaggle/input/chh-ola/train.csv')
df

,ID,vendor+AF8-id,pickup+AF8-loc,drop+AF8-loc,driver+AF8-tip,mta+AF8-tax,distance,pickup+AF8-time,drop+AF8-time,num+AF8-passengers,toll+AF8-amount,payment+AF8-method,rate+AF8-code,stored+AF8-flag,extra+AF8-charges,improvement+AF8-charge,total+AF8-amount
0,0,1,170.0,233.0,1.83,0.5,0.70,04/04/2017 05:59:43 PM,04/04/2017 06:05:04 PM,1.0,0,1.0,1.0,N,1,0.3,9.13
1,1,2,151.0,243.0,3.56,0.5,4.64,04/03/2017 07:03:34 PM,04/03/2017 07:20:04 PM,1.0,0,1.0,1.0,N,1,0.3,21.36
2,2,2,68.0,90.0,1.5,0.5,1.29,04/03/2017 03:06:13 PM,04/03/2017 03:12:30 PM,2.0,0,1.0,1.0,N,0,0.3,8.8
3,3,2,142.0,234.0,1.5,0.5,2.74,04/04/2017 08:10:52 AM,04/04/2017 08:27:00 AM,1.0,0,1.0,1.0,N,0,0.3,14.8
4,4,2,238.0,238.0,0,0.5,0.45,04/05/2017 02:02:59 PM,04/05/2017 02:05:41 PM,6.0,0,2.0,1.0,N,0,0.3,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048570,1048570,1,48.0,50.0,1,0.5,1.40,04/06/2017 11:17:09 PM,04/06/2017 11:25:04 PM,1.0,0,1.0,1.0,N,0.5,0.3,9.8
1048571,1048571,1,237.0,166.0,0,0.5,3.60,04/06/2017 10:58:47 AM,04/06/2017 11:22:04 AM,1.0,0,2.0,1.0,N,0,0.3,18.3
1048572,1048572,1,164.0,90.0,2.35,0.5,1.70,04/06/2017 10:03:42 PM,04/06/2017 10:17:04 PM,1.0,0,1.0,1.0,N,0.5,0.3,14.15
1048573,1048573,2,264.0,264.0,0,0.5,1.80,04/06/2017 06:43:22 PM,04/06/2017 06:56:09 PM,4.0,0,2.0,1.0,N,1,0.3,11.8


# Step 1) - EDA and Feature Engineering

In [5]:
# Summary statistics of the numerical columns
print(df.describe())

                 ID  pickup+AF8-loc  drop+AF8-loc      distance  \
count  1.048575e+06    1.048574e+06  1.048574e+06  1.048574e+06   
mean   5.242870e+05    1.648085e+02  1.626675e+02  2.859832e+00   
std    3.026977e+05    6.579282e+01  6.953073e+01  3.709541e+00   
min    0.000000e+00    1.000000e+00  1.000000e+00  0.000000e+00   
25%    2.621435e+05    1.250000e+02  1.130000e+02  9.300000e-01   
50%    5.242870e+05    1.620000e+02  1.620000e+02  1.600000e+00   
75%    7.864305e+05    2.330000e+02  2.330000e+02  2.900000e+00   
max    1.048574e+06    2.650000e+02  2.650000e+02  1.138000e+02   

       num+AF8-passengers  payment+AF8-method  rate+AF8-code  
count        1.048574e+06        1.048574e+06   1.048574e+06  
mean         1.590696e+00        1.317943e+00   1.043368e+00  
std          1.253700e+00        4.853266e-01   5.621114e-01  
min          0.000000e+00        1.000000e+00   1.000000e+00  
25%          1.000000e+00        1.000000e+00   1.000000e+00  
50%          1.000

In [6]:
# Check the data types and missing values in each column
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1048575 entries, 0 to 1048574
Data columns (total 17 columns):
ID                        1048575 non-null int64
vendor+AF8-id             1048575 non-null object
pickup+AF8-loc            1048574 non-null float64
drop+AF8-loc              1048574 non-null float64
driver+AF8-tip            1048573 non-null object
mta+AF8-tax               1048574 non-null object
distance                  1048574 non-null float64
pickup+AF8-time           1048574 non-null object
drop+AF8-time             1048574 non-null object
num+AF8-passengers        1048574 non-null float64
toll+AF8-amount           1048573 non-null object
payment+AF8-method        1048574 non-null float64
rate+AF8-code             1048574 non-null float64
stored+AF8-flag           1048574 non-null object
extra+AF8-charges         1048574 non-null object
improvement+AF8-charge    1048573 non-null object
total+AF8-amount          1048573 non-null object
dtypes: float64(6), int64(1), ob

In [ ]:
# Pairplot to visualize relationships between numerical columns (be cautious if the dataset is large)
sns.pairplot(df) # takes about 10 mins to run
plt.show()

/opt/conda/lib/python3.6/site-packages/numpy/lib/histograms.py:824: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= first_edge)
/opt/conda/lib/python3.6/site-packages/numpy/lib/histograms.py:825: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= last_edge)


NameError: name 'plt' is not defined

In [ ]:
# Correlation heatmap to identify correlations between numerical columns
correlation_matrix = df.corr()
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Distribution plot for a numerical column (e.g., distance)
sns.histplot(df['distance'], kde=True)
plt.title('Distribution of Distance')
plt.show()


In [ ]:

# Bar plot for a categorical column (e.g., payment_Method)
sns.countplot(x='payment_Method', data=df)
plt.title('Payment Method Distribution')
plt.show()

In [ ]:
# Box plot to visualize the distribution and outliers of a numerical column (e.g., total_amount)
sns.boxplot(x='total_amount', data=df)
plt.title('Total Amount Distribution')
plt.show()


In [ ]:
# Scatter plot to observe the relationship between two numerical columns (e.g., distance vs. total_amount)
sns.scatterplot(x='distance', y='total_amount', data=df)
plt.title('Distance vs. Total Amount')
plt.show()

In [ ]:
# Bar plot for a categorical column with hue (e.g., payment_Method with vendor_id as hue)
sns.countplot(x='payment_Method', hue='vendor_id', data=df)
plt.title('Payment Method Distribution with Vendor ID')
plt.show()

In [ ]:
sns.violinplot(x='vendor_id', y='driver_tip', data=df)
plt.title('Tip Distribution by Vendor ID')
plt.show()

In [ ]:
sns.boxplot(x='payment_Method', y='total_amount', data=df)
plt.title('Fare Amount by Payment Method')
plt.show()

In [ ]:
sns.boxplot(x='vendor_id', y='total_amount', hue='payment_Method', data=df)
plt.title('Fare Amount by Vendor ID with Payment Method')
plt.show()


In [ ]:
# Extract the Geographic Segment - Analyze popular pickup and drop locations
pickup_counts = df['pickup_loc'].value_counts()
drop_counts = df['drop_loc'].value_counts()

# Visualize popular pickup locations
plt.figure(figsize=(10, 6))
sns.barplot(x=pickup_counts.index, y=pickup_counts.values)
plt.title('Popular Pickup Locations')
plt.xlabel('Location ID')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

# Visualize popular drop locations
plt.figure(figsize=(10, 6))
sns.barplot(x=drop_counts.index, y=drop_counts.values)
plt.title('Popular Drop Locations')
plt.xlabel('Location ID')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.show()

# Extract the Behavioral Segment - Analyze fare amounts, extra charges, and tipping behavior
plt.figure(figsize=(10, 6))
sns.boxplot(x='rate_code', y='total_amount', data=df)
plt.title('Fare Amount by Rate Code')
plt.xlabel('Rate Code')
plt.ylabel('Total Amount')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='payment_Method', y='total_amount', data=df)
plt.title('Fare Amount by Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Total Amount')
plt.show()

plt.figure(figsize=(10, 6))
sns.boxplot(x='payment_Method', y='driver_tip', data=df)
plt.title('Tip Distribution by Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Driver Tip')
plt.show()

# Analyze spending patterns based on payment methods
payment_spending = df.groupby('payment_Method')['total_amount'].mean().reset_index()

plt.figure(figsize=(8, 5))
sns.barplot(x='payment_Method', y='total_amount', data=payment_spending)
plt.title('Average Spending by Payment Method')
plt.xlabel('Payment Method')
plt.ylabel('Average Total Amount')
plt.show()


# Step 2) Cleaning the data using bash is faster than in Python.

In [ ]:
!sed 's/\+AF8-//g' /kaggle/input/chh-ola/train.csv > train.csv
!sed 's/_//g' /kaggle/input/chh-ola/test.csv > test.csv

# Step 3) Write some utility functions

In [ ]:
class Ut:
  @staticmethod
  def to_timestamp(dt):
    return dt_parse(dt, dayfirst=False).timestamp()

  @staticmethod
  def flag_to_num(vl):
    if vl == 'N':
      return 0
    else:
      return 1
  
  @staticmethod
  def to_float(vl):
    try:
      if type(vl) == type('str'):
        idx = vl.find('-')
        if idx != -1:
          txt = vl.split('-')
          return float(txt[1])
      return float(vl)
    except:
      print(vl)
      return float(0)

  @staticmethod
  def rmse(predictions, targets):
    """Data convertion and RMSE calculation"""
    return np.sqrt(mean_squared_error(np.exp(predictions), np.exp(targets)))

# Step 4) Data preprocessing

In [ ]:
# Load training data file
train_set = pd.read_csv('train.csv', low_memory=False, dtype=str)
train_set.dropna(inplace=True)
train_set.reset_index(drop=True, inplace=True)

# Load testing data file
test_set = pd.read_csv('test.csv', low_memory=False, dtype=str)
test_set['totalamount'] = 0

# Create a column to distinguish one from the other
train_set['PROPOSITO'] = 1
test_set['PROPOSITO'] = 0

# Concatenate all data into one dataframe
all_set = pd.concat([train_set, test_set], ignore_index=True)

# Delete train_set and test_set to free memory
del(train_set)
del(test_set)

# MTA Tax should be the same in all rides
all_set['mtatax'] = 0.5

# Data conversion
# Y or N to 1 or 0
all_set['storedflag'] = all_set['storedflag'].apply(Ut.flag_to_num)
# Conversion to datetime format
all_set['pickuptime'] = all_set['pickuptime'].apply(Ut.to_timestamp)
all_set['droptime'] = all_set['droptime'].apply(Ut.to_timestamp)
# Conversion to float
all_set['drivertip'] = all_set['drivertip'].apply(Ut.to_float)
all_set['mtatax'] = all_set['mtatax'].apply(Ut.to_float)
all_set['tollamount'] = all_set['tollamount'].apply(Ut.to_float)
all_set['extracharges'] = all_set['extracharges'].apply(Ut.to_float)
all_set['improvementcharge'] = all_set['improvementcharge'].apply(Ut.to_float)
all_set['totalamount'] = all_set['totalamount'].apply(Ut.to_float)

# Step 5) Feature creation: **total time** and **taxes**.

In [ ]:
# Total time
all_set['totaltime'] = all_set['droptime'] - all_set['pickuptime']
# All Taxes
all_set['taxes'] = all_set['drivertip'] + all_set['mtatax'] + all_set['tollamount'] + all_set['extracharges'] + all_set['improvementcharge']

# Step 6) Feature analysis and type convertion.

In [ ]:
features_cat = ['vendorid', 'paymentmethod', 'ratecode', 'storedflag']
features_num = ['drivertip', 'pickuploc', 'droploc', 'mtatax', 'distance', 'pickuptime', 'droptime', 'numpassengers', 
                'tollamount', 'extracharges', 'improvementcharge', 'totalamount', 'totaltime', 'taxes']
target = 'totalamount'

# Numerical features will be converted to float.
for col in features_num:
    all_set[col] = all_set[col].astype(float)

# Categorical features will be converted to float then to string.    
for col in features_cat:
    all_set[col] = all_set[col].astype(float)
    all_set[col] = all_set[col].astype(str)

all_set['PROPOSITO'] = all_set['PROPOSITO'].astype(int)
all_set['ID'] = all_set['ID'].astype(int)

# Step 7) Split data into *train_df* and *test_df*

In [ ]:
# Convert categorical variable into dummy/indicator variables
all_dum = pd.get_dummies(all_set)

train_df = all_dum[all_dum['PROPOSITO'] == 1].copy()
test_df = all_dum[all_dum['PROPOSITO'] == 0].copy()

del(all_dum)

train_df.drop(columns=['PROPOSITO'], inplace=True)
test_df.drop(columns=['PROPOSITO'], inplace=True)

# Remove rows where pickup loc is equal to drop loc
train_df = train_df[train_df['pickuploc'] != train_df['droploc']]
# Drop rows where total amount is 0, since there are no free rides
train_df = train_df[train_df['totalamount'] > 0]
test_df.drop(columns=[target], inplace=True)

# Step 8) Normalize data
Normalized data provide better results on the model. 
* The data will be normalized using **sklearn.preprocessing.Normalizer()**. 
* The targets will be normalized using **numpy.log**. 

In [ ]:
# Alongside hyperparameter searching, I also did a feature searching to check 
# which combination would provide better results
features_to_keep = [
  'taxes',
  'pickuploc',
  'ratecode_2.0',
  'ratecode_1.0',
  'ratecode_5.0',
  'storedflag_0.0',
  'ratecode_4.0',
  'totaltime',
  'ratecode_3.0',
  'droploc',
  'numpassengers',
  'distance',
  'storedflag_1.0',
  'vendorid_2.0',
  'paymentmethod_1.0',
  'vendorid_1.0',
  'paymentmethod_2.0'
  ]

X = train_df[features_to_keep].copy()
y = train_df[target].copy()

normalizer = Normalizer()
norm_X = normalizer.fit_transform(X)
y = np.log(y)

# Step 9) Hyperparameter searching

### a) GridSearchCV

In [ ]:
if False:
  params = {
      'colsample_bytree':[0.9], 
      'gamma':[0.3],
      'max_depth': [9],  
      'min_child_weight':[2], 
      'subsample':[0.9],
      'n_estimators': [50],
      'objective': ['reg:squarederror'],
      'n_jobs': [8],
      }

  # Initialize XGB and GridSearch
  eval_model = xgb.XGBRegressor(nthread=-1) 

  grid = GridSearchCV(eval_model, params, cv=2)
  grid.fit(train_X, train_y)

  pred_y = grid.predict(test_X)
  print('RMSE Test = ', Ut.rmse(pred_y, test_y))

  print(grid)
  print(grid.best_params_)

### b) RandomizedSearchCV

In [ ]:
if False:
  params = {
      'min_child_weight': st.randint(2, 9), 
      'gamma': st.uniform(0.1, 0.9),  
      'subsample': st.uniform(0.1, 0.9),
      'colsample_bytree': st.uniform(0.1, 0.9), 
      'max_depth': st.randint(3, 9),
      'n_estimators': [50],
      'objective': ['reg:squarederror'],
      # 'n_jobs': [8],
      }

  eval_model = xgb.XGBRegressor(nthread=-1) 

  grid = RandomizedSearchCV(eval_model, params, cv=2, n_jobs=1, n_iter=10)
  grid.fit(train_X, train_y)

  pred_y = grid.predict(test_X)
  print('RMSE Test = ', Ut.rmse(pred_y, test_y))

  print(grid)

### c) Feature searching

In [ ]:
# Besides hyperparameter searching, it is also important to see how the results change 
# when we remove features that have a poor correlation to the "total amount".
if False:
    train_correlation = train_df.corr()
    train_correlation = train_correlation['totalamount'].apply(abs)
    train_correlation = train_correlation.sort_values(na_position='first')
    train_correlation = pd.DataFrame(train_correlation).reset_index()
    train_correlation.dropna(inplace=True)
    
    best_rmse = math.inf
    best_idx = 0

    features_to_analyse = train_correlation['index'].unique()
    for idx in range(len(features_to_analyse) - 1):
        features = features_to_analyse[idx:]
        dX = train_df[features].copy()
        dy = train_df[target].copy()

        normalizer = Normalizer()
        norm_dX = normalizer.fit_transform(dX)
        dy = np.log(dy)

        train_dX, test_dX, train_dy, test_dy = train_test_split(norm_dX, dy, test_size=0.2, random_state=42)

        params = {
            'objective': 'reg:squarederror',
            'n_estimators': 100,
            'subsample': 0.9, 
            'min_child_weight': 1, 
            'max_depth': 9,
            'gamma': 0.3, 
            'colsample_bytree': 0.9,
            'n_jobs': 8,
            'verbose_eval':'False',
        }
        model = xgb.XGBRegressor(**params)
        model.fit(train_dX,train_dy)

        pred_dy = model.predict(test_dX)
        print('Features cut = ', idx, ' Resulting RMSE = ', Ut.rmse(pred_dy, test_dy))
        
        if best_rmse > rmse_res:
            best_rmse = rmse_res
            best_idx = idx

    print('Features cut = ', best_idx, ' Resulting RMSE = ', best_rmse)
    features_to_keep = features_to_analyse[best_idx:]

# Step 10) Model training

In [ ]:
train_X, test_X, train_y, test_y = train_test_split(norm_X, y, test_size=0.2, random_state=42)

# These hyperparameters are the result of the searchings of Step 9
params = {
    'objective': 'reg:squarederror',
    'n_estimators': 1000,
    'subsample': 0.9, 
    'min_child_weight': 1, 
    'max_depth': 9,
    'gamma': 0.3, 
    'colsample_bytree': 0.9,
    'n_jobs': 8,
    'verbose_eval':'False',
}
model = xgb.XGBRegressor(**params)
model.fit(train_X,train_y)

pred_y = model.predict(test_X)
print('RMSE Test = ', Ut.rmse(pred_y, test_y))

# Step 11) Prediction

In [ ]:
real_X = normalizer.transform(test_df[features_to_keep].copy())

model = xgb.XGBRegressor(**params)
model.fit(norm_X, y)

predictions = np.exp(model.predict(real_X))

In [ ]:
# Have a glimpse of the predictions
pd.DataFrame(predictions).head(20)

# Step 12) Saving the results

In [ ]:
result = []
for idx in range(test_df.shape[0]):
  result.append([idx, predictions[idx]])
result = pd.DataFrame(result, columns=['ID', 'total_amount'])
result.to_csv('result.csv', index=False)

In [ ]:
# Delete temp files so they're not taken as results by mistake
!rm train.csv
!rm test.csv